# XML example and exercise
****
+ study examples of accessing nodes in XML tree structure  
+ work on exercise to be completed and submitted
****
+ reference: https://docs.python.org/2.7/library/xml.etree.elementtree.html
+ data source: http://www.dbis.informatik.uni-goettingen.de/Mondial
****

In [274]:
from xml.etree import ElementTree as ET

## XML example

+ for details about tree traversal and iterators, see https://docs.python.org/2.7/library/xml.etree.elementtree.html

In [275]:
document_tree = ET.parse( './data/mondial_database_less.xml' )

In [276]:
# print names of all countries
for child in document_tree.getroot():
    print child.find('name').text

Albania
Greece
Macedonia
Serbia
Montenegro
Kosovo
Andorra


In [277]:
# print names of all countries and their cities
for element in document_tree.iterfind('country'):
    print '* ' + element.find('name').text + ':',
    capitals_string = ''
    for subelement in element.getiterator('city'):
        capitals_string += subelement.find('name').text + ', '
    print capitals_string[:-2]

* Albania: Tirana, Shkodër, Durrës, Vlorë, Elbasan, Korçë
* Greece: Komotini, Kavala, Athina, Peiraias, Peristeri, Acharnes, Patra, Kozani, Kerkyra, Ioannina, Thessaloniki, Iraklio, Chania, Ermoupoli, Rhodes, Tripoli, Lamia, Chalkida, Larissa, Volos, Mytilini, Karyes
* Macedonia: Skopje, Kumanovo
* Serbia: Beograd, Novi Sad, Niš
* Montenegro: Podgorica
* Kosovo: Prishtine
* Andorra: Andorra la Vella


****
## XML exercise

Using data in 'data/mondial_database.xml', the examples above, and refering to https://docs.python.org/2.7/library/xml.etree.elementtree.html, find

1. 10 countries with the lowest infant mortality rates
2. 10 cities with the largest population
3. 10 ethnic groups with the largest overall populations (sum of best/latest estimates over all countries)
4. name and country of a) longest river, b) largest lake and c) airport at highest elevation

In [278]:
tree = ET.parse( './data/mondial_database.xml' )
root = tree.getroot()

## 10 Countries with the lowest infant mortality rates

In [279]:
import pandas as pd
df=pd.DataFrame({'country':[],'infant_mortality':[]})
for cou in root.findall('country'):
    name=None
    for na in cou.findall("name"):
        name=na.text
        break
    infm=None
    for im in cou.findall("infant_mortality"):
        infm=float(im.text)
        break
    if name and infm:
        df=df.append(pd.DataFrame({'country':[name],'infant_mortality':[infm]}),ignore_index=True)
df.sort_values(['infant_mortality']).head(10)

,country,infant_mortality
36,Monaco,1.81
90,Japan,2.13
109,Bermuda,2.48
34,Norway,2.48
98,Singapore,2.53
35,Sweden,2.60
8,Czech Republic,2.63
72,Hong Kong,2.73
73,Macao,3.13
39,Iceland,3.15


## 10 cities with the largest population

In [280]:
df=pd.DataFrame({'country':[],'city':[],'population':[]})
for cou in root.findall('country'):
    name=None
    for na in cou.findall("name"):
        name=na.text
        break
    for city in cou.findall("*/city"):
        cname=None
        popu=None
        for cna in city.findall("name"):
            cname=cna.text
            break
        for pop in city.findall("population"):
            popu=float(pop.text)
        if name and cname and popu:
            df=df.append(pd.DataFrame({'country':[name],'city':[cname],'population':[popu]}),ignore_index=True)
df.sort_values(['population'],ascending=False).head(10)

,city,country,population
1192,Shanghai,China,22315474.0
673,Istanbul,Turkey,13710512.0
1353,Mumbai,India,12442373.0
421,Moskva,Russia,11979529.0
1191,Beijing,China,11716620.0
2303,São Paulo,Brazil,11152344.0
1193,Tianjin,China,11090314.0
915,Guangzhou,China,11071424.0
1399,Delhi,India,11034555.0
918,Shenzhen,China,10358381.0


## 10 ethnic groups with the largest overall populations

In [281]:
import numpy as np
df=pd.DataFrame({})
for cou in root.findall('country'):
    popu=None
    for pop in cou.findall("population"):
        popu=float(pop.text)
    ethnic=None
    percent=None
    dfe=pd.DataFrame({})
    for eth in cou.findall("ethnicgroup"):
        ethnic=eth.text
        if 'percentage' in eth.attrib:
            percent=float(eth.attrib['percentage'])/100.0
        if popu and ethnic and percent:
            dfe[ethnic]=[int(percent*popu)]
    df=df.append(dfe,ignore_index=True)
df=df.fillna(0)
df=df.astype("int64")
df=df.apply(np.cumsum).iloc[-1].sort_values(ascending=False).to_frame().head(10)
df=df.rename(columns=dict(zip(df.columns.values,["Population"])))
df

,Population
Han Chinese,1245058800
Indo-Aryan,871815583
European,494872201
African,318325104
Dravidian,302713744
Mestizo,157734349
Bengali,146776916
Russian,131856989
Japanese,126534212
Malay,121993548


## Name and country of a) longest river, b) largest lake and c) airport at highest elevation

In [282]:
#---Make country database---#
df_cou=pd.DataFrame({})
for cou in root.findall('country'):
    code=None
    name=None
    if 'car_code' in cou.attrib:
        code=cou.attrib['car_code']
    for na in cou.findall("name"):
        name=na.text
        break
    if length and name:
        df_cou=df_cou.append(pd.DataFrame({'name':[name],'code':[code]}),ignore_index=True)

def get_top_landmark(landmark,category,best=True):
    df_best=pd.DataFrame({})
    for lan in root.findall(landmark):
        name=None
        val=None
        cou=None
        if 'country' in lan.attrib:
            cou=lan.attrib['country'].split(" ")
        for na in lan.findall("name"):
            if na.text:
                name=na.text
                break
        for cat in lan.findall(category):
            if cat.text:
                val=float(cat.text)
                break
        if val and name:
            df_best=df_best.append(pd.DataFrame({str(landmark)+'_name':[name],'country':[cou],category:[val]}),ignore_index=True)

    def country_list(list,db):
        for i in range(len(list)):
            try:
                list[i]=db[db['code']==list[i]]['name'].values[0]
            except:
                pass
        return list

    df_best['country']=df_best['country'].apply(lambda x:country_list(x,df_cou))
    df_best=df_best.sort_values([category],ascending=not best).head(1)
        
    return df_best

In [283]:
get_top_landmark("river","length",best=True)

,country,length,river_name
174,"[Colombia, Brazil, Peru]",6448.0,Amazonas


In [284]:
get_top_landmark("lake","area",best=True)

,area,country,lake_name
54,386400.0,"[Russia, Azerbaijan, Kazakhstan, Iran, Turkmen...",Caspian Sea


In [285]:
get_top_landmark("airport","elevation",best=True)

,airport_name,country,elevation
80,El Alto Intl,[Bolivia],4063.0
